In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

In [5]:
def sam_analysis():
    print('=======================================')

    print()
    print('사회계정행렬 선택: ')
    print('1. 거시 사회계정행렬 \n 2. 거시 사회계정행렬(국산) \n 3. 미시 사회계정행렬(산업 세분화) \n 4. 미시 사회계정행렬(산업 세분화: 국산) \n 5. 미시 사회계정행렬(최종) \n 6. 미시 사회계정행렬(최종: 국산))')
    print()
    i = int(input('승수 분석을 진행할 SAM의 종류를 선택해주세요. : '))
    
    
    if i == 1:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM.xlsx')
        
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM.xlsx')
        print('데이터 불러오기 완료')
        print('========================================')
        name = list(data.columns[7:])
    elif i == 2:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM(국산).xlsx')
        
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM(국산).xlsx')
        print('데이터 불러오기 완료')
        print('========================================')
        name = list(data.columns[7:])
    elif i == 3:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬 RAS 적용.xlsx')
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬 RAS 적용.xlsx')
        name = list(data.columns[27:])
    elif i == 4:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(국산) RAS 적용.xlsx')
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(국산) RAS 적용.xlsx')
        name = list(data.columns[27:])

    elif i == 5:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종) RAS 적용.xlsx')        
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종) RAS 적용.xlsx')        
        name = list(data.columns[28:])
    elif i == 6:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종 국산) RAS 적용.xlsx')
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종 국산) RAS 적용.xlsx')
        name = list(data.columns[28:])
        
    else:
        print('번호를 잘못 입력했습니다. 함수를 다시 실행해주세요. ')
        print('========================================')
    
    data.rename(columns={'Unnamed: 0':'구분'},inplace=True)
    data.set_index('구분', inplace=True)
    data.fillna(0, inplace=True)
    data.replace(' ', 0, inplace=True)
    data = data.astype('float')

    
    print()
    
    print('데이터 전처리 및 불러오기 완료')
    print('========================================')
    
    
    s11 = data.loc[:'정부', :'정부'].values.copy()

    s12 = data.loc[:'정부', '고정자본형성':'수입'].values.copy()
    
    s21 = data.loc['고정자본형성':'수입', :'정부'].values.copy()
    
    s22 = data.loc['고정자본형성':'수입', '고정자본형성':'수입'].values.copy()
    
    
    y1_r = data.loc['총계', :'정부'].values.reshape(1, -1).copy()
    
    y2_r = data.loc['총계', '고정자본형성':'수입'].values.reshape(1, -1).copy()
        

    y1_c = data.loc[:'정부', '총계'].values.reshape(-1, 1).copy()
    y2_c = data.loc['고정자본형성':'수입', '총계'].values.reshape(-1, 1).copy()
    
    
    # 투입계수 도출 
    
    h11 = s11 / y1_r
    h12 = s12 / y2_r
    h21 = s21 / y1_r
    h22 = s22 / y2_r
    
        
    # 승수행렬
    SAM_multiplier = np.linalg.inv(np.identity(h11.shape[0])-h11)   
    print()
    print('승수 행렬 : ')
    print(SAM_multiplier)
    print()
    
    print(name)
    print()
    added_name = input("값을 추가하고 싶은 계정 이름을 입력해주세요: ")
    
    name_index = name.index(added_name)
    
    print('=======================')
    added_value = float(input('추가하실 금액을 입력해주세요: '))
    
    
    y2_r[0][name_index] = y2_r[0][name_index] + added_value
    
    result_vector = np.dot(np.dot(SAM_multiplier, h12), y2_r.transpose()).transpose()    

    update_metrix = h11 * result_vector    
    
    
    data.loc[:'정부', :'정부'] = update_metrix

    data.drop(columns=['총계'], index=['총계'], inplace=True)
    
    
    print()
    print('승수분석을 완료하였습니다. ')
    print('=======================')
    display(data)
    
    return SAM_multiplier, data

In [6]:
sam_multiplier, data = sam_analysis()


사회계정행렬 선택: 
1. 거시 사회계정행렬 
 2. 거시 사회계정행렬(국산) 
 3. 미시 사회계정행렬(산업 세분화) 
 4. 미시 사회계정행렬(산업 세분화: 국산) 
 5. 미시 사회계정행렬(최종) 
 6. 미시 사회계정행렬(최종: 국산))

승수 분석을 진행할 SAM의 종류를 선택해주세요. : 5

데이터 전처리 및 불러오기 완료

승수 행렬 : 
[[1.09291634e+00 1.12590347e-03 4.93130095e-02 3.82477739e-02
  2.34436569e-02 4.43120603e-02 3.57177710e-02 8.87811360e-02
  3.34770672e-02 3.42969340e-02 2.47268783e-02 3.76850729e-02
  4.28789191e-02 4.74220481e-02 5.17048538e-02 3.96032568e-02
  2.61480173e-02 3.51891776e-02 2.79169202e-02 3.28227674e-02
  4.71857039e-02 5.18219212e-02 2.16560192e-02 6.40126559e-03
  5.18219212e-02 6.40126559e-03 3.30563090e-02]
 [5.71663706e-02 1.00204287e+00 1.16832381e-01 6.01636719e-02
  3.96873335e-01 7.90923422e-02 6.72129742e-02 7.50612731e-02
  5.36530210e-02 4.99104993e-02 4.00696625e-02 6.31794432e-02
  6.07138609e-02 7.26698740e-02 8.11326359e-02 6.88042240e-02
  5.26930333e-02 5.64243309e-02 4.31127486e-02 4.77361330e-02
  6.19938547e-02 6.52201238e-02 2.83647691e-02 9.72719102e-03
  6.5220

값을 추가하고 싶은 계정 이름을 입력해주세요: SIT 고정자본형성
추가하실 금액을 입력해주세요: 3601.8

승수분석을 완료하였습니다. 


,농림어업,광업,제조업,도매 및 소매업,"전기, 가스, 증기 및 수도사업",건설업,운수업,숙박 및 음식점업,정보 및 통신업,금융 및 보험업,...,기업,정부,고정자본형성,SIT 고정자본형성,간접세,법인세,소득세,관세,수출,수입
구분,,,,,,,,,,,,,,,,,,,,,
농림어업,4480.643156,4.324052,39092.632490,49.138932,10.365867,381.216487,21.094418,9484.876820,19.264103,36.145174,...,0.000000,0.000000,384.460515,0.000000,0.000,0.000,0.000,0.000,1079.658410,0.000
광업,0.560902,2.182356,110962.180074,0.000000,37756.573904,293.977364,0.000000,4.207127,0.000000,0.000000,...,0.000000,0.000000,1989.331803,0.000000,0.000,0.000,0.000,0.000,112.782142,0.000
제조업,17783.000111,660.399025,840451.492008,12885.122097,13895.608911,103291.550636,41924.245355,58801.212617,8228.612704,4043.782411,...,0.000000,0.000000,163247.363913,0.000000,0.000,0.000,0.000,0.000,621195.288721,0.000
도매 및 소매업,1438.468908,35.250731,43231.047603,4430.753298,747.255036,5128.187944,1779.339732,8480.000867,5054.900197,587.716851,...,0.000000,0.000000,9518.423648,0.000000,0.000,0.000,0.000,0.000,22359.609575,0.000
"전기, 가스, 증기 및 수도사업",916.859163,108.707882,38039.209371,2404.375605,16216.603343,1054.379610,2611.276614,5172.404161,1266.244009,1438.625558,...,0.000000,1010.741574,0.000000,0.000000,0.000,0.000,0.000,0.000,1106.918555,0.000
건설업,81.928286,16.666725,1209.523659,187.756881,541.100335,87.401349,216.649665,338.784879,232.089128,92.558411,...,0.000000,0.000000,260512.298209,0.000000,0.000,0.000,0.000,0.000,192.357391,0.000
운수업,1029.003380,639.835184,47906.936570,13523.848995,1824.086226,2460.553476,24272.500281,1169.871904,1808.645087,906.315714,...,0.000000,0.000000,697.564655,0.000000,0.000,0.000,0.000,0.000,35167.066671,0.000
숙박 및 음식점업,622.803042,98.173696,11744.019118,7550.353235,933.777313,1144.706993,2629.882934,1869.220668,1944.751349,4733.968072,...,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.000,11199.537836,0.000
정보 및 통신업,199.133960,15.700099,4114.564188,5867.855158,372.704297,408.539648,1503.250229,1292.713844,13123.650255,7008.370659,...,0.000000,0.000000,19336.276274,0.000000,0.000,0.000,0.000,0.000,8154.382106,0.000


In [7]:
data.to_excel('SCN2.xlsx')